<a href="https://colab.research.google.com/github/habrev/Ethiomart/blob/task-4/notebooks/fine_tune_xlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate

# Specify the models to compare
models = [
    {"name": "xlm-roberta-base", "path": "xlm-roberta-base"},
    {"name": "distilbert-base-multilingual-cased", "path": "distilbert-base-multilingual-cased"},
    {"name": "bert-base-multilingual-cased", "path": "bert-base-multilingual-cased"}
]

# Load dataset and metric
dataset = load_dataset("@mertteka_labeled_data.conll")
eval_metric = load_metric("seqeval")

# Preprocess dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

  # Define compute_metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(-1)
    true_labels = p.label_ids
    results = eval_metric.compute(predictions=predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }


# Loop through each model
results = []
for model_info in models:
    print(f"Fine-tuning {model_info['name']}...")

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_info["path"])
    model = AutoModelForTokenClassification.from_pretrained(model_info["path"], num_labels=len(dataset["train"].features["ner_tags"].feature.names))

    # Tokenize dataset
    tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{model_info['name']}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs/{model_info['name']}",
        logging_steps=10,
        save_total_limit=2
    )

    # Define Trainer
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


    # Fine-tune model
    trainer.train()

    # Evaluate model
    metrics = trainer.evaluate()
    results.append({"model": model_info["name"], "metrics": metrics})

# Compare Results
print("\nModel Comparison:")
for result in results:
    print(f"Model: {result['model']} | F1: {result['metrics']['eval_f1']} | Accuracy: {result['metrics']['eval_accuracy']} | Loss: {result['metrics']['eval_loss']}")


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: '@mertteka_labeled_data.conll'.

In [11]:
import pandas as pd

def load_conll_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # If the line is not empty
                tokens = line.split()  # Split by whitespace
                if len(tokens) >= 2:  # Ensure there are at least 2 columns (token and label)
                    token, label = tokens[0], tokens[1]  # First token is the word, second is the label
                    data.append((token, label))  # Append as a tuple

    # Create a DataFrame with appropriate columns
    df = pd.DataFrame(data, columns=['Token', 'Label'])
    return df

In [12]:
# Usage
conll_file_path = '@mertteka_labeled_data.conll'
df = load_conll_to_dataframe(conll_file_path)

df.head()

,Token,Label
0,ይሄንን,O
1,ተጭነው,O
2,ያድርጉ፣,O
3,ቤተሰብ,O
4,ይሁኑ,O


In [13]:

# Split the dataset into training and test portions
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=200, random_state=21)


In [ ]:
import pandas as pd
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Map labels to IDs
label_list = df['Label'].unique().tolist()
label_map = {label: idx for idx, label in enumerate(label_list)}
num_labels = len(label_map)

# Encode tokens and labels
def encode_data(df):
    tokens = []
    labels = []
    for _, group in df.groupby((df['Label'] != df['Label'].shift()).cumsum()):
        tokenized_input = tokenizer(list(group['Token']),
                                    is_split_into_words=True,
                                    padding='max_length',
                                    truncation=True,
                                    # max_length=128,
                                    return_tensors='pt')
        tokens.append(tokenized_input)
        label_ids = [label_map[label] for label in group['Label']]
        label_ids = label_ids + [label_map['O']] * (tokenized_input['input_ids'].shape[1] - len(label_ids))  # Padding
        labels.append(torch.tensor(label_ids))
    return tokens, labels

tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
train_df, val_df = train_test_split(df_train, test_size=0.2)
train_tokens, train_labels = encode_data(train_df)
val_tokens, val_labels = encode_data(val_df)

# Create a dataset class
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokens[idx]['input_ids'].squeeze(),
            'attention_mask': self.tokens[idx]['attention_mask'].squeeze(),
            'labels': self.labels[idx]
        }

train_dataset = NERDataset(train_tokens, train_labels)
val_dataset = NERDataset(val_tokens, val_labels)

# Model and Training Setup
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
    logging_dir='./logs',
    eval_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
